In [1]:
pip install tflearn

In [2]:
import numpy as np
import tensorflow as tf
import tflearn
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [4]:
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [5]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
  intents = json.load(json_data)

Processing the Intents.....


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through intents to convert them to words, classes, documents ang ignore_words.....")
for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

Looping through intents to convert them to words, classes, documents ang ignore_words.....


In [8]:
print("Stemming, Lowering and removing duplicates.....")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)

Stemming, Lowering and removing duplicates.....
26 documents
8 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [9]:
print("Creating the data for our model.....")
training = []
output = []
print("Creating a List(Empty) for output.....")
output_empty = [0] * len(classes)
print("Creating Training Set, Bag of words for our model.....")
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])


Creating the data for our model.....
Creating a List(Empty) for output.....
Creating Training Set, Bag of words for our model.....


In [10]:
training[1][1]

[0, 1, 0, 0, 0, 0, 0, 0]

In [11]:
import tensorflow as tf

In [12]:
print("Shuffling Randomly and Converting into Numpy Array for faster processing.....")
random.shuffle(training)
training = np.array(training)
print("Creating Train and Test lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Build Neural Network for out chatbot to be contextual.....")
print("Resetting graph data.....")
tf.compat.v1.reset_default_graph()


Shuffling Randomly and Converting into Numpy Array for faster processing.....
Creating Train and Test lists.....
Build Neural Network for out chatbot to be contextual.....
Resetting graph data.....


In [13]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training.....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training.....


In [14]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [15]:
print("Training the Model.....")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.....")
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.72435 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.72435 - acc: 0.8975 -- iter: 24/26
Training Step: 4000  | total loss: 0.67877 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.67877 - acc: 0.9077 -- iter: 26/26
--
Saving the Model.....
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
print("Pickle is also saved.....")
pickle.dump( {'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y}, open ("training_data", "wb") )

Pickle is also saved.....


In [17]:
print("Loading pickle.....")
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
with open('intents.json') as json_data:
  intents = json.load(json_data)
print("Loading the Model.....")
model.load('./model.tflearn')

Loading pickle.....
Loading the Model.....
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [18]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
  return sentence_words

def bow(sentence, words, show_details=False):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)
  for s in sentence_words:
    for i, w in enumerate(words):
      if w == s:
        bag[i] = 1
        if show_details:
          print("found in bag: %s" % w)
  return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
  results = model.predict([bow(sentence, words)])[0]
  results = [[i,r] for i, r in enumerate(results) if r>ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append((classes[r[0]],r[1]))
  return return_list  

def response(sentence, userID='123', show_details=False):
  results = classify(sentence)
  if results:
    while results:
      for i in intents['intents']:
        if i['tag'] == results[0][0]:
          return print(random.choice(i['responses']))
      results.pop(0)



ERROR_THRESHOLD = 0.25


In [ ]:
while True:
  input_data = input("enter your query- ")
  answer = response(input_data)
  answer

enter your query- hii
Good to see you again
enter your query- open today
our hours are 9am-9pm everyday
enter your query- which mopeds do you have
We rent yamaha, Piaggio and vespa mopeds
enter your query- do you accept mastercard
We accept most major credit cards
enter your query- that's helpful
Anytime!
enter your query- thank you
Anytime!
enter your query- goodbye
have a nice day
